<a href="https://colab.research.google.com/github/cs16b108/IB_MDDP/blob/master/Helper_Functions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
from os.path import isfile, isdir, join
from pathlib import Path
from sklearn import mixture
import math
import numpy as np
import random
from scipy.spatial import distance
import scipy.io.wavfile as wav
import pickle
import time
import copy

In [4]:
#Give complete path of vuv_frame.npy
#File name and directory path
fileName = 'ES2002c'
drivePath = '/content/drive/My Drive/IB_Diarization/'
vuv_frames = np.load(join(drivePath,'save_files', fileName+'_vuv.npy'))

In [ ]:
################################################
#### Generate Segment file (.scp) for IDIAP ####
################################################
fobj = open(fileName+'.scp', 'a')
L = len(vuv_frames)
i = 0
while i<L:
  if vuv_frames[i] == 1:
    j = i
    cnt = 1
    while j+1<L and vuv_frames[j+1] == 1:
      cnt += 1
      j += 1
    curLine = fileName+'_'+str(i*3)+'_'+str(j*3)+'='+fileName+'.fea['+str(i*3)+','+str(j*3)+']\n'
    f.writelines(curLine)
    i = j
  i += 1
fobj.close()

In [6]:
###############################################
#### Generate rttm from amicorpus segments ####
###############################################
import xml.etree.ElementTree as ET
import numpy as np
ns = {'nite':"http://nite.sourceforge.net/", 'id':fileName+".A.segs"}
alls = ['A','B','C','D']
Vec = np.zeros((3000000,))
diarizeReference = []

def parse(S, diarizeReference, speaker_num):	
	tree = ET.parse('/content/drive/My Drive/IB_Diarization/ami_public_manual_1.6.2/segments/'+fileName+'.'+S+'.segments.xml')
	root = tree.getroot()
  
	for seg in root.findall('segment'):
		st = seg.get('transcriber_start')
		end = seg.get('transcriber_end')
		diarizeReference.append([float(st), float(end), 'speaker_'+str(int(speaker_num))])
		# lineStr = 'SPEAKER ' + 'meeting' + ' ' + channelId + ' ' + str(onset) + ' ' + str(duration) + ' <NA> <NA> ' \
    #                 + 'speaker_'+str(int(finalSegments[idx-1])) + ' <NA> <NA>\n'
    #   fileObj.writelines(lineStr)
	return diarizeReference

d=0
for speaker_num, S in enumerate(alls):
	diarizeReference = parse(S,diarizeReference, speaker_num)
	d+=1
diarizeReference.sort()
print(diarizeReference)

fileObj = open(fileName+'_actual.rttm', 'a')
for d in diarizeReference:
  lineStr = 'SPEAKER ' + 'meeting' + ' ' + '1' + ' ' + str(d[0]) + ' ' + str(d[1]-d[0]) + ' <NA> <NA> ' \
                    + d[2] + ' <NA> <NA>\n'
  fileObj.writelines(lineStr)
fileObj.close()

[[0.0, 8.837, 'speaker_0'], [0.6, 0.956, 'speaker_3'], [4.135, 5.294, 'speaker_1'], [4.998, 15.166, 'speaker_3'], [6.312, 7.716, 'speaker_1'], [12.71, 13.268, 'speaker_0'], [12.993, 13.686, 'speaker_1'], [15.056, 20.64, 'speaker_1'], [16.262, 16.626, 'speaker_0'], [21.936, 28.944, 'speaker_1'], [28.864, 30.352, 'speaker_0'], [30.272, 57.772, 'speaker_1'], [32.4, 33.008, 'speaker_3'], [34.064, 34.84, 'speaker_3'], [57.772, 62.4, 'speaker_1'], [60.987, 61.765, 'speaker_0'], [63.024, 63.529, 'speaker_0'], [63.573, 65.456, 'speaker_3'], [63.84, 93.744, 'speaker_1'], [75.232, 75.913, 'speaker_3'], [77.069, 77.568, 'speaker_3'], [82.496, 83.952, 'speaker_3'], [84.261, 84.608, 'speaker_0'], [89.152, 90.836, 'speaker_3'], [91.552, 93.248, 'speaker_3'], [94.447, 94.785, 'speaker_3'], [98.64, 99.504, 'speaker_1'], [100.704, 102.176, 'speaker_1'], [107.789, 130.48, 'speaker_1'], [114.736, 115.221, 'speaker_0'], [125.392, 126.894, 'speaker_3'], [131.302, 132.864, 'speaker_3'], [134.256, 137.552, '

In [ ]:
########################################
#### Generate SCP from PRAAT Output ####
########################################
f = open('IS1000a_Mix-Headset_25', 'r') #Give praat output file name
fw = open('IS1000a.scp', 'a') #Give scp file name to be generated
lst = f.readlines()
# scp_vec = np.zeros((227976), dtype=int)
for i in range(13, len(lst)-2):
  if lst[i+2] == '"sounding"\n':
    l = int(float(lst[i])*100)
    r = int(float(lst[i+1])*100)
    curLine = 'IS1000a_'+str(l)+'_'+str(r)+'=IS1000a.fea['+str(l)+','+str(r)+']\n' ##Change according to the name of scp file
    fw.writelines(curLine)     
f.close()
fw.close()